Os dados ex6-train.csv representam um problema de regressão. A última coluna 
coluna é o valor a ser aproximado.

Use várias técnicas para fazer a regressão ( SVM regressão, gbm, rf, redes neurais, 
knn, gaussian regression, e outras mesmo que não tenhamos visto em aula). A métrica 
séra MAE - erro absoluto médio (nao erro quadrado!).

O relatório deve conter a sua exploracão de pelo menos 2 das téecnicas de regressao 
(mais o pre-processamento, se for o caso), os hiperparâmetros tentados e o erro (MAE). 
O relatório valerá 60% da nota.

Rode o seu melhor regressor nestes dados, e submita também o resultado do valor predito, 
um por linha na mesma ordem dos dados. Note que é um resultado por linha.

Eu avaliarei o MAE do seu regressor nos resultados corretos. 40% restante da nota sera 
competitiva: as submissões no topo 10% com menos MAE receberão o 10 nessa parte e as 
submissões nos últimos 10% (maiores MAE) receberão 0.

Note que o dataset é grande (50.000 linhas e 77 atributos). Talvez não valha a pena fazer 
a exploração dos algoritimos e a busca de hiperparâmetros no dataset inteiro.

ATENCAO: é preciso submeter 2 arquivos, o pdf com o relatório (60% da nota) e um csv com os 
valores computados para os dados em ex6-test.csv

# Nested cross validation para escolher os melhores parâmetros

In [ ]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import random

# Escolhe os melhores parâmetros para os regressores
def nested_cross_validation(param_grid, regressor, dataset, target):
    # O Grid Search já é criado com valores de 3-fold e StratifiedKfold
    # por default
    
    gridSearch = GridSearchCV(
            estimator = regressor, param_grid = param_grid, cv = 3, 
            scoring='neg_mean_absolute_error', n_jobs=8)
    gridSearch.fit(dataset, target)
    # Faço a validaçao cruzada com 5-fold
    cross_val_predict(gridSearch, X = dataset, y = target, cv = 5, 
                      n_jobs=8)
    # Retorno os melhores parametros encontrados pelo Grid Search
    return gridSearch.best_params_ 

# Lê o arquivo de entrada com 50000 linhas e separa os valores de entrada e de saída

In [ ]:
file = '/home/mauricio/projects/MO444/lista 6/ex6-train.csv'
# Arquivo completo
dataset_full = pd.read_csv(file).values
target_full = dataset_full[:,-1]
X_train_full = np.delete(dataset_full,-1,1)

# Cria um dataset com 20% dos dados, escolhidos aleatoriamente e separa os valores de Entrada e de saída

In [ ]:
# Conta as linhas
num_lines = sum(1 for l in open(file))
#  20% dos dados
size = int(num_lines / 5)
# Cria dataset com 20% dos dados
skip_idx = random.sample(range(1, num_lines), num_lines - size)
dataset_sample = pd.read_csv(file, skiprows=skip_idx).values
target_sample = dataset_sample[:,-1]
X_train_sample = np.delete(dataset_sample,-1,1)

# Cria 2 datatsets com valores standardizados e com PCA (full e com 20% dos dados). O PCA foi feito com 98% dos dados porque foi o que conseguimos os melhores valores de erro (MAE) nos regressores

In [ ]:
standard_scaler = StandardScaler()
X_train_scaler_full = standard_scaler.fit_transform(X_train_full)
X_train_scaler_sample = standard_scaler.fit_transform(X_train_sample)
# PCA para Reduzirmos dimensão dos dados para manter 98% da variância
pca = PCA(n_components=0.98)
X_train_pca_full = pca.fit_transform(X_train_scaler_full)
X_train_pca_sample = pca.fit_transform(X_train_scaler_sample)

# Regressão Linear com Arquivo completo (50000 linhas)

In [ ]:
linear_regression = LinearRegression()
# Sem PCA
linear_model = linear_regression.fit(X_train_full, target_full)
linear_regression_mae_error = mean_absolute_error(
    target_full, linear_model.predict(X_train_full))
print("Linear Regression MAE[%0.3f]" %linear_regression_mae_error)

# Com PCA
linear_model = linear_regression.fit(X_train_pca_full, target_full)
linear_regression_mae_error = mean_absolute_error(
    target_full, linear_model.predict(X_train_pca_full))
print("Linear Regression PCA MAE[%0.3f]" %linear_regression_mae_error)

Linear Regression MAE[0.107] 

Linear Regression PCA MAE[0.107]

# SVR com amostragem (10000 linhas)

In [ ]:
# SVR - Nested Cross Validation
param_grid_svr = [{'C': [2**(-5), 2**(0), 2**(5), 2**(10)], 
                    'gamma':[ 2**(-15),2**(-10),2**(-5),2**(0),2**(5)],
                    'epsilon':[0.1, 1.5]}]
svr_rbf = SVR(kernel='rbf')

# Sem PCA
svr_best_param = nested_cross_validation(param_grid_svr, svr_rbf, 
                                         X_train_sample, target_sample)
print("SVR Best parameters")
print(svr_best_param)

# Com PCA
svr_best_param = nested_cross_validation(param_grid_svr, svr_rbf, 
                                         X_train_pca_sample, target_sample)
print("SVR Best parameters PCA")
print(svr_best_param)

SVR Best parameters
{'C': 1024, 'epsilon': 0.1, 'gamma': 3.0517578125e-05}

SVR Best parameters PCA
{'C': 32, 'epsilon': 0.1, 'gamma': 3.0517578125e-05}

# Com a escolha dos parâmetros realizada, faço o treinamento e calculo o MAE no conjunto completo

In [ ]:
# Sem PCA
svr_rbf = SVR(kernel='rbf', C=1024, gamma=3.0517578125e-05, epsilon=0.1)
svr_rbf.fit(X_train_full, target_full)
svr_mae_error = mean_absolute_error(target_full, 
                                    svr_rbf.predict(X_train_full))
print("SVR MAE [%0.3f] "%svr_mae_error)

# Com PCA
svr_rbf = SVR(kernel='rbf', C=32, gamma=3.0517578125e-05, epsilon=0.1)
svr_rbf.fit(X_train_pca_full, target_full)
svr_mae_error = mean_absolute_error(target_full, 
                                    svr_rbf.predict(X_train_pca_full))
print("SVR MAE PCA [%0.3f] "%svr_mae_error)

SVR MAE [0.081] 

SVR MAE PCA [0.099]

# KernelRidge com amostragem (10000 linhas)

In [ ]:
# KernelRidge - Nested Cross Validation
param_grid_ridge = [{'alpha': [2**(-5), 2**(0), 2**(5), 2**(10)], 
                    'gamma':[ 2**(-15),2**(-10),2**(-5),2**(0),2**(5)]}]
kernel_ridge = KernelRidge(kernel='rbf')

# Sem PCA
ridge_best_param = nested_cross_validation(param_grid_ridge, kernel_ridge, 
                                           X_train_sample, target_sample)
print("kernel Ridge Best parameters")
print(ridge_best_param)

# Com PCA
ridge_best_param = nested_cross_validation(param_grid_ridge, kernel_ridge, 
                                           X_train_pca_sample, 
                                           target_sample)
print("kernel Ridge Best parameters PCA")
print(ridge_best_param)

kernel Ridge Best parameters
{'alpha': 0.03125, 'gamma': 3.0517578125e-05}

kernel Ridge Best parameters PCA
{'alpha': 0.03125, 'gamma': 3.0517578125e-05}

# Com a escolha dos parâmetros realizada, faço o treinamento e calculo o MAE. Para o kernel Ridge não foi possível fazer o treinamento no conjunto completo, por falta de memória, então foi feito com metade dos dados

In [ ]:
# 50% dos dados
size_half = int(num_lines / 2)
skip_idx = random.sample(range(1, num_lines), num_lines - size_half)
dataset_sample_half = pd.read_csv(file, skiprows=skip_idx).values
target_sample_half = dataset_sample_half[:,-1]

# Sem PCA
X_train_sample_half = np.delete(dataset_sample_half,-1,1)

# Com PCA
X_train_scaler_sample_half = standard_scaler.fit_transform(
    X_train_sample_half)
X_train_pca_sample_half = pca.transform(X_train_scaler_sample_half)
# Dataset full com PCA e mesmo nr de dimensões usados no treino
X_train_full_pca_ridge =  pca.transform(X_train_full)


In [ ]:
kernel_ridge = KernelRidge(kernel='rbf', alpha=0.03125, 
                           gamma=3.0517578125e-05)
# Sem PCA
kernel_ridge.fit(X_train_sample_half, target_sample_half)
ridge_mae_error = mean_absolute_error(target_sample_half, 
                                      kernel_ridge.predict(
                                          X_train_sample_half))
print("Kernel Ridge MAE [%0.3f] "%ridge_mae_error)

# Com PCA
kernel_ridge.fit(X_train_pca_sample_half, target_sample_half)
ridge_mae_error = mean_absolute_error(target_full, 
                                      kernel_ridge.predict(
                                          X_train_full_pca_ridge))
print("kernel Ridge MAE PCA [%0.3f] "%ridge_mae_error)

Kernel Ridge MAE [0.086]

Kernel Ridge MAE PCA [0.307]  

# Random Forest com amostragem (10000 linhas)

In [ ]:
# Random Forest Nested Cross Validation

# Sem PCA
param_grid_rf = [{'n_estimators': [ 100, 200, 400, 800], 
                 'max_features': [10, 30, 50, 77]}]
random_forest = RandomForestRegressor()
rf_best_param = nested_cross_validation(param_grid_rf, random_forest, 
                                        X_train_sample, target_sample)
print("Random Forest Best parameters")
print(rf_best_param)

# com PCA
param_grid_rf = [{'n_estimators': [ 100, 200, 400, 800], 
                 'max_features': [8, 12, 17, 21]}]
rf_best_param = nested_cross_validation(param_grid_rf, random_forest, 
                                        X_train_pca_sample, target_sample)
print("Random Forest Best parameters PCA")
print(rf_best_param)

Random Forest Best parameters
{'max_features': 30, 'n_estimators': 400}

Random Forest Best parameters PCA
{'max_features': 17, 'n_estimators': 800}

In [ ]:
# Sem PCA 
random_forest = RandomForestRegressor(n_estimators=400, max_features=30)
random_forest.fit(X_train_full, target_full)
rf_mae_error = mean_absolute_error(target_full, 
                                   random_forest.predict(X_train_full))
print("Random Forest MAE [%0.3f] "%rf_mae_error)

# Com PCA 
random_forest = RandomForestRegressor(n_estimators=800, max_features=17)
random_forest.fit(X_train_pca_full, target_full)
rf_mae_error = mean_absolute_error(target_full, 
                                   random_forest.predict(X_train_pca_full))
print("Random Forest MAE PCA [%0.3f] "%rf_mae_error)

Random Forest MAE [0.030]

Random Forest MAE PCA [0.034]

# Conclusões:
- Foram feitos testes (não colocados todos aqui) usando separadamente 
a standardização e o PCA, mas os resultados não foram os melhores
- Colocamos os resultados com os dados sem processamento e com 
stardardização+PCA para efeitos de comparação
- O melhor resultado (com menor MAE) foi o Random Forest (MAE = 0.030)
- Portanto é ele que iremos utilizar para gerar as predições no arquivo de 
testes

In [ ]:
fileTest = '/home/mauricio/projects/MO444/lista 6/ex6-test.csv'
# Test dataset
dataset_test = pd.read_csv(fileTest).values
random_forest = RandomForestRegressor(n_estimators=400, max_features=30)
random_forest.fit(X_train_full, target_full)
y_prediction = random_forest.predict(dataset_test)
pd.DataFrame(y_prediction).to_csv('prediction.csv', header=False)

# Arquivo gerado prediction.csv